## Neural Networks for Patches

In [214]:
#global imports
import numpy as np
import tensorflow as tf
import tensorflow.contrib.learn as skflow
from sklearn.preprocessing import normalize
from sklearn.metrics import mean_squared_error, r2_score
import time
from datetime import timedelta
import itertools

In [2]:
from util import PatchesDataLoader, PatchesSKLearner

### Data Imports

In [33]:
#data_path = '/home/jacky/ws/patches/data/'
data_path = '/mnt/wd_ssd/projects/deep_patches/data/patches_06_11_16'
pdl = PatchesDataLoader(0.25, data_path, [i for i in range(1)], by_objs=False)

In [4]:
def one_hot(types, labels):
    result = np.zeros((labels.shape[0], len(types)))
    for i in range(labels.shape[0]):
        result[i][types[labels[i]]] = 1
    return result        

In [34]:
fc_tr_1h = one_hot([0, 1], pdl.labels['fc']['tr'])
fc_t_1h = one_hot([0, 1], pdl.labels['fc']['t'])

In [6]:
pfc_10_tr = pdl.labels['pfc_10']['tr'].reshape(-1, 1)
pfc_10_t = pdl.labels['pfc_10']['t'].reshape(-1, 1)

KeyError: 'pfc_10'

In [84]:
def get_cropped_X(pdl, dim):
    
    def crop(img, dim):
        big_dim = int(np.sqrt(img.shape[0]))
        img = img.reshape(big_dim, big_dim)
        mid = big_dim // 2
        delta = dim // 2

        small_img = img[mid - delta : mid + delta + 1, mid - delta : mid + delta + 1]

        return small_img.reshape(dim*dim)

    cropped_w1 = np.array([crop(pdl._raw_data['w1_proj'][i,:], 3) for i in range(len(pdl._raw_data['w1_proj']))])
    cropped_w2 = np.array([crop(pdl._raw_data['w2_proj'][i,:], 3) for i in range(len(pdl._raw_data['w2_proj']))])

    cropped_w1_tr, cropped_w1_t, _ = pdl.split_train_test(cropped_w1, 0.25, pdl.indices)
    cropped_w2_tr, cropped_w2_t, _ = pdl.split_train_test(cropped_w1, 0.25, pdl.indices)

    X = pdl.get_partial_train_data(('moment_arms', 'patch_ori'))

    X['tr'] = np.c_[X['tr'], cropped_w1_tr, cropped_w2_tr]
    X['t'] = np.c_[X['t'], cropped_w1_t, cropped_w2_t]
    
    return X

In [85]:
X = get_cropped_X(pdl, 3)

### DNN For Classification

In [73]:
batch_size = 100

In [89]:
clf = skflow.TensorFlowDNNClassifier(hidden_units=[50, 50, 50], n_classes=2, batch_size=batch_size, steps=5000,
                                    optimizer="Adam", learning_rate=0.001, dropout=0.25, continue_training=True)

In [90]:
start = time.time()
clf.fit(X['tr'], pdl.labels['fc']['tr'])
end = time.time()
print 'took {0}s'.format(end - start)

Step #100, epoch #4, avg. train loss: 0.67778
Step #200, epoch #9, avg. train loss: 0.62479
Step #300, epoch #13, avg. train loss: 0.60075
Step #400, epoch #18, avg. train loss: 0.57387
Step #500, epoch #22, avg. train loss: 0.55217
Step #600, epoch #27, avg. train loss: 0.54057
Step #700, epoch #31, avg. train loss: 0.52769
Step #800, epoch #36, avg. train loss: 0.51661
Step #900, epoch #40, avg. train loss: 0.51055
Step #1000, epoch #45, avg. train loss: 0.50031
Step #1100, epoch #50, avg. train loss: 0.49292
Step #1200, epoch #54, avg. train loss: 0.48682
Step #1300, epoch #59, avg. train loss: 0.47828
Step #1400, epoch #63, avg. train loss: 0.47869
Step #1500, epoch #68, avg. train loss: 0.47181
Step #1600, epoch #72, avg. train loss: 0.46332
Step #1700, epoch #77, avg. train loss: 0.46078
Step #1800, epoch #81, avg. train loss: 0.45502
Step #1900, epoch #86, avg. train loss: 0.44796
Step #2000, epoch #90, avg. train loss: 0.44984
Step #2100, epoch #95, avg. train loss: 0.43965
Ste

In [79]:
PatchesSKLearner.print_accuracy(clf, X['tr'], pdl.labels['fc']['tr'], X['t'], pdl.labels['fc']['t'], "DNN Classification")

DNN Classification train accuracy 0.856011183597, test accuracy 0.758041958042


In [237]:
from sklearn.externals import joblib
help(clf.save)


Help on method save in module tensorflow.contrib.learn.python.learn.estimators.base:

save(self, path) method of tensorflow.contrib.learn.python.learn.estimators.dnn.TensorFlowDNNClassifier instance
    Saves checkpoints and graph to given path.
    
    Args:
        path: Folder to save model to.



### Grid Search CV on Classification

In [223]:
class LoopTimingForecaster:
    
    def __init__(self, n):
        self._n = n
        self._i = 0
        self._start = None
        self._temp_start = None
        self._end = None
        
        self._times = []
        
    def _check_end(self):
        if self._end is not None:
            raise Exception("Can't perform this operation because end has already been called")
                
    def report_forecast(self):
        self._check_end()
        if len(self._times) == 0:
            print "No data on timing yet. Can't perform forecast"
        else:
            remaining_steps = self._n - self._i
            mean_time = sum(self._times) / len(self._times)
            remaining_time = mean_time * remaining_steps
            
            msg = "Remaining time: {0}, steps: {1}. Time per step: {2}".format(timedelta(seconds=remaining_time),
                                                                               remaining_steps, timedelta(seconds=mean_time))
            print msg
            
    def record_loop_start(self):
        cur_time = time.time()
        self._check_end()
        if self._start is None:
            self._start = cur_time
            print "Starting loop timer"
        self._temp_start = cur_time
    
    def record_loop_end(self):
        cur_time = time.time()
        self._check_end()
        
        if self._temp_start is None:
            raise Exception("Can't end loop timer because it hasn't been started!")
        
        self._times.append(cur_time - self._temp_start)
        self._temp_start = None
        self._i += 1
        
    def record_end(self):
        self._end = time.time()

In [224]:
class SKFlowGridSearchCV:
    
    def __init__(self, estimator_instantiator, params, cv=0.25, random_state=None):
        self._estimator_instantiator = estimator_instantiator
        self._cv = cv
        self._random_state = random_state
        self._best_score = None
        self._best_params = None
        self._best_estimator = None
        self._scores = None
        self._params = params
        self._params_grid = None
        self._generate_params_grid()
        
    def _generate_params_grid(self):
        self._params_grid = []
        keys = self._params.keys()
        options = self._params.values()
        options_combos = itertools.product(*options)

        for options_combo in options_combos:
            param = {}
            for i in range(len(keys)):
                param[keys[i]] = options_combo[i]

            self._params_grid.append(param)

    def fit(self, X, y):
        self._scores = []
        self._best_score = float('-inf')
        self._best_params = None
        self._best_estimator = None
        
        X_tr, X_v, y_tr, y_v = train_test_split(X, y, test_size=self._cv)
        ltf = LoopTimingForecaster(len(self._params_grid))
        for params in self._params_grid:
            ltf.record_loop_start()
            estimator = self._estimator_instantiator().set_params(**params)
            estimator.fit(X_tr, y_tr)
            score = estimator.score(X_v, y_v)
            self._scores.append((params, score))
            
            if score > self._best_score:
                self._best_score = score
                self._best_params = params
                self._estimator = estimator
            else:
                del estimator
            ltf.record_loop_end()
            ltf.report_forecast()
                
    @property
    def best_params_(self):
        return self._best_params
    
    @property
    def best_score_(self):
        return self._best_score
    
    @property
    def grid_scores_(self):
        return self._scores
    
    @property
    def best_estimator_(self):
        return self._best_estimator
    
    def predict(self, X, y):
        return self._best_estimator.predict(X, y)
    
    def score(self, X, y):
        return self._best_estimator.score(X, y)

In [225]:
def get_hidden_units(layer_nums, layer_sizes):
    all_hidden_unit_combos = []
    for layer_num in layer_nums:
        hidden_unit_combos = [combo for combo in itertools.permutations(layer_sizes, layer_num)]
        all_hidden_unit_combos.extend(hidden_unit_combos)
    return all_hidden_unit_combos

In [228]:
layer_sizes = [50, 100, 200, 500]
layer_nums = [1, 2, 3, 4]
hidden_units = get_hidden_units(layer_nums, layer_sizes)

nn_fc_params_grid = {
 'batch_size': [100],
 'continue_training': [False],
 'dropout': [0, 0.1, 0.25, 0.5, 0.7],
 'hidden_units': hidden_units,
 'learning_rate': [1e-3, 1e-1, 1e-5],
 'n_classes': [2],
 'optimizer': ['Adam', 'SGD', 'Adagrad'],
 'steps': [5000],
 'verbose': [0]
}

In [ ]:
nn_fc_cv = SKFlowGridSearchCV(lambda:skflow.TensorFlowDNNClassifier(hidden_units=[100], n_classes=2), nn_fc_params_grid)
nn_fc_cv.fit(X['tr'], pdl.labels['fc']['tr'])

Starting loop timer
Remaining time: 14:07:35.945858, steps: 2879. Time per step: 0:00:17.664448
Remaining time: 13:00:17.137099, steps: 2878. Time per step: 0:00:16.267247
Remaining time: 12:42:07.948154, steps: 2877. Time per step: 0:00:15.894316
Remaining time: 12:28:21.803226, steps: 2876. Time per step: 0:00:15.612588
Remaining time: 12:36:29.532995, steps: 2875. Time per step: 0:00:15.787664
Remaining time: 12:24:02.476830, steps: 2874. Time per step: 0:00:15.533221
Remaining time: 12:00:43.661054, steps: 2873. Time per step: 0:00:15.051744
Remaining time: 11:43:54.788831, steps: 2872. Time per step: 0:00:14.705706
Remaining time: 11:34:34.378313, steps: 2871. Time per step: 0:00:14.515632
Remaining time: 11:21:56.217421, steps: 2870. Time per step: 0:00:14.256522
Remaining time: 11:12:49.339986, steps: 2869. Time per step: 0:00:14.070875
Remaining time: 11:05:29.342962, steps: 2868. Time per step: 0:00:13.922365
Remaining time: 10:58:58.453753, steps: 2867. Time per step: 0:00:13

In [230]:
print nn_fc_cv.best_params_

{'continue_training': False, 'optimizer': 'Adam', 'verbose': 0, 'dropout': 0.25, 'batch_size': 100, 'steps': 5000, 'n_classes': 2, 'hidden_units': (50, 100, 200), 'learning_rate': 0.001}


In [231]:
clf.predict_proba

0.776536312849


### DNN For Regression

In [19]:
reg = skflow.TensorFlowDNNRegressor(hidden_units=[500, 200], batch_size=batch_size, steps=5000,
                                    optimizer="Adam", learning_rate=0.001, dropout=0.25, continue_training=True)

In [21]:
reg.fit(pdl.tr, pdl.labels['pfc_10']['tr'])

KeyError: 'pfc_10'

In [22]:
PatchesSKLearner.print_accuracy(reg, pdl.tr, pdl.labels['pfc_10']['tr'], pdl.t, pdl.labels['pfc_10']['t'], "DNN Regression PFC")

DNN Regression PFC train accuracy 0.775900502456, test accuracy 0.747592332814


In [23]:
PatchesSKLearner.print_mse(reg, pdl.tr, pdl.labels['pfc_10']['tr'], pdl.t, pdl.labels['pfc_10']['t'], "DNN Regression PFC mse")

DNN Regression PFC mse train mse 0.0228855780185, test mse 0.0254127871325


In [56]:
all_ferrari = np.r_[pdl.labels['ferrari']['tr'], pdl.labels['ferrari']['t']]
n_ferrari_tr = pdl.labels['ferrari']['tr'].shape[0]
ferrari_mean, ferrari_std = np.mean(all_ferrari), np.std(all_ferrari)
all_ferrari_normalized = (all_ferrari - ferrari_mean) / ferrari_std

In [57]:
ferrari_normalized_tr = all_ferrari_normalized[:n_ferrari_tr]
ferrari_normalized_t = all_ferrari_normalized[n_ferrari_tr:]

In [60]:
reg_ferrari = skflow.TensorFlowDNNRegressor(hidden_units=[500, 200], batch_size=batch_size, steps=5000,
                                    optimizer="Adam", learning_rate=0.001, dropout=0.25, continue_training=True)

In [61]:
reg_ferrari.fit(X['tr'], ferrari_normalized_tr)

Step #100, epoch #4, avg. train loss: 0.48840
Step #200, epoch #9, avg. train loss: 0.43664
Step #300, epoch #13, avg. train loss: 0.41896
Step #400, epoch #18, avg. train loss: 0.40423
Step #500, epoch #22, avg. train loss: 0.39277
Step #600, epoch #27, avg. train loss: 0.38682
Step #700, epoch #31, avg. train loss: 0.37682
Step #800, epoch #36, avg. train loss: 0.36997
Step #900, epoch #40, avg. train loss: 0.36687
Step #1000, epoch #45, avg. train loss: 0.35941
Step #1100, epoch #50, avg. train loss: 0.35468
Step #1200, epoch #54, avg. train loss: 0.34959
Step #1300, epoch #59, avg. train loss: 0.35008
Step #1400, epoch #63, avg. train loss: 0.34232
Step #1500, epoch #68, avg. train loss: 0.33779
Step #1600, epoch #72, avg. train loss: 0.32967
Step #1700, epoch #77, avg. train loss: 0.33234
Step #1800, epoch #81, avg. train loss: 0.32764
Step #1900, epoch #86, avg. train loss: 0.32361
Step #2000, epoch #90, avg. train loss: 0.32120
Step #2100, epoch #95, avg. train loss: 0.31596
Ste

TensorFlowDNNRegressor(batch_size=100, clip_gradients=5.0, config=None,
            continue_training=True, dropout=0.25, hidden_units=[500, 200],
            learning_rate=0.001, n_classes=0, optimizer='Adam', steps=5000,
            verbose=1)

In [63]:
PatchesSKLearner.print_accuracy(reg_ferrari, X['tr'], ferrari_normalized_tr, X['t'], 
                                ferrari_normalized_t, "DNN Regression Ferrari Canny Normalized")

DNN Regression Ferrari Canny Normalized train accuracy 0.611247086652, test accuracy 0.243951896547


In [64]:
PatchesSKLearner.print_mse(reg_ferrari, X['tr'], ferrari_normalized_tr, X['t'], 
                                ferrari_normalized_t, "DNN Regression Ferrari Canny Normalized mse")

DNN Regression Ferrari Canny Normalized mse train mse 0.388062952178, test mse 0.759942829183


In [66]:
ferrari_preds_normalized_tr = reg_ferrari.predict(X['tr'])
ferrari_preds_normalized_t = reg_ferrari.predict(X['t'])

ferrari_preds_tr = (ferrari_preds_normalized_tr * ferrari_std) + ferrari_mean
ferrari_preds_t = (ferrari_preds_normalized_t * ferrari_std) + ferrari_mean

In [30]:
ferrari_preds_tr = (ferrari_preds_normalized_tr * ferrari_std) + ferrari_mean
ferrari_preds_t = (ferrari_preds_normalized_t * ferrari_std) + ferrari_mean

In [67]:
print "{0} train {1}, test {2}".format("DNN Regression Ferrari Canny", 
                                               r2_score(pdl.labels['ferrari']['tr'], ferrari_preds_tr), 
                                               r2_score(pdl.labels['ferrari']['t'], ferrari_preds_t))

DNN Regression Ferrari Canny train 0.611247094, test 0.243951889254


In [68]:
print "{0} train mse {1}, test mse {2}".format("DNN Regression Ferrari Canny", 
                                               mean_squared_error(pdl.labels['ferrari']['tr'], ferrari_preds_tr), 
                                               mean_squared_error(pdl.labels['ferrari']['t'], ferrari_preds_t))

DNN Regression Ferrari Canny train mse 3.48378707123e-08, test mse 6.82229278498e-08
